# Serial Femtosecond Crystallography at European XFEL

## Experimental setup
In a typical SFX experiment at the SPB/SFX (and also the FXE) instrument, microcrystalline sample is delivered by a liquid jet system - either acqueous solution delivered into a vacuum chamber with jet speeds up to 120 m/s, or more viscous solutions delivered to an interaction region at atmospheric pressure. The jet stream is aligned to the X-ray beam such that sample is hit by FEL X-ray pulses perpendicularly at a rate of typically 1.1 MHz (900 ns intervals of femtosecond pulses, up to 352 pulses in a train, 10 trains per second), and with the detector at the same repetition rate for recordings, i.e. taking a total of 3520 image frames per second. The crystal hit rate largely depends on the jet speed, and lies in the range of 1% to 10% for most experiments. It is much higher for the slower viscous jets.

![XFEL_crystallography_setup](img/XFEL_crystallography.png)


The goal of the experiment is to reconstruct the electron density within the crystal unit cell (resp. asymmetric unit) so that a structural model of the protein can be built into it.

For example, we will be working with the data collected with the hen egg white lysozyme:

![lysozyme_3D_small](img/lysozyme_3D_small.png)

## Exploring the data

First of all, lets load some useful libraries:

In [ ]:
from subprocess import check_output

import numpy as np
import h5py
import matplotlib.pyplot as plt

Data collected in each experiment are stored in the dedicated proposal folders. To easily find the proposal (and run) folder we provide a command line tool `findxfel`:

```shell
findxfel 700000 30 --proc
```

in this example `--proc` stands for 'processed', or calibrated by offline calibration pipeline, data.

In [ ]:
findxfel_command = "findxfel 700000 30 --proc"
p700000_r0030_path = check_output(findxfel_command.split()).decode('utf-8').strip()
print(f"\nData from the example proposal 700000 run 30 is stored at:\n\n{p700000_r0030_path}\n")

We can take a look inside of this folder with the `ls` command.

In [ ]:
p700000_r0030_files = check_output(f"ls {p700000_r0030_path}".split()).decode('utf-8')
for file in p700000_r0030_files.split()[:4]:
    if 'CORR-R0097-AGIPD' in file:
        print(file)
print("...")
for file in p700000_r0030_files.split()[61:64]:
    if 'CORR-R0097-AGIPD' in file:
        print(file)
print("...")

Name of the data file, e.g. `CORR-R0097-AGIPD03-S00001.h5`, contains some useful information:
- `CORR` stands for corrected (processed/calibrated) data
- `R0097` represents the original run number during which the data have been collected
- `AGIPD03` means module 3 of the AGIPD detector, in total this detector has 16 modules numbered from 0 to 15
- `S00001` reperesents a sequence, basically an order in which datafiles have been written

We can peek into structure of one of the files with the `h5glance` command:

```shell
h5glance $(findxfel 700000 30 --proc)/CORR-R0097-AGIPD07-S00012.h5
```

In [ ]:
p700000_r0030_1file = f"{p700000_r0030_path}/CORR-R0097-AGIPD04-S00000.h5"
p700000_r0030_structure = check_output(f"h5glance {p700000_r0030_1file}".split()).decode('utf-8')
print(p700000_r0030_structure)

Take a closer look at `/INSTRUMENT/SPB_DET_AGIPD1M-1/DET/4CH0:xtdf/image/data` - this is an array of our data:
```text
data [float32: 28416 × 512 × 128]
```

There `28416` is the number of collected data frames in this files, each `512px × 128px`. Lets visualize data from 1 frame:

In [ ]:
with h5py.File(p700000_r0030_1file, 'r') as h5_file:
    agipd_data_arr_p4 = h5_file['/INSTRUMENT/SPB_DET_AGIPD1M-1/DET/4CH0:xtdf/image/data'][95]

plt.figure(figsize=(12, 3))
plt.imshow(np.swapaxes(agipd_data_arr_p4, 0, 1), origin='lower', interpolation='none', vmin=0, vmax=3000)

## Detector geometry

To work with the multimodular detectors geometry data analysis group provide an `extra_geom` library:

In [ ]:
from extra_geom import AGIPD_1MGeometry, LPD_1MGeometry

Geometry file contains information on the spatial position of each detector module:

In [ ]:
agipd_geom_file = "../geom/agipd_p700000_r0030_v01.geom"
agipd_geom = AGIPD_1MGeometry.from_crystfel_geom(agipd_geom_file)
agipd_geom.inspect()

Such geometry object allows to plot whole detector image, but it requires data from all modules arranged in one array. There are 3 obvious options on how we can proceed:
- read data for each module from the `proc` files and arrange it into a single array manually
- use `extra_data` module to read and stack detector data from the `proc` files
- or use `extra_data` to generate and write virtual dataset with links to the `proc` data arranged into a single array

We will use the third option. For this run in the terminal following commands:

```shell
cd ~/hercules_SFX_2023/data
./make_vds_proc.sh
```

`make_vds_proc.sh` script executes `extra-data-make-virtual-cxi`, a command line interface of `extra_data` for generating such VDS files.

Let's take a look inside the generated file:

```shell
h5glance p700000_r0030_proc.cxi
```

or:

In [ ]:
p700000_r0030_vds_file = "../data/p700000_r0030_proc.cxi"
p700000_r0030_vds_structure = check_output(f"h5glance {p700000_r0030_vds_file}".split()).decode('utf-8')
print(p700000_r0030_vds_structure)

`/entry_1/instrument_1/detector_1/data` is an array with all `639616` data frames from `16` detector modules, each `512px × 128px`.

In [ ]:
with h5py.File(p700000_r0030_vds_file, 'r') as vds_file:
    agipd_data_arr = vds_file['/entry_1/data_1/data'][95]

agipd_geom.plot_data_fast(agipd_data_arr, vmin=0, vmax=3000)

## Mask bad detector pixels

By exploring the detector image above one might notice that it contains not only water scattering and lysozyme break peaks, but also some bright misbehaving pixels. Most of these bad pixels can be covered with a mask generated by the offline calibration pipeline, which is stored under `/INSTRUMENT/SPB_DET_AGIPD1M-1/DET/4CH0:xtdf/image/mask` in the `proc` data or `/entry_1/instrument_1/detector_1/mask` in the VDS file.

In [ ]:
with h5py.File(p700000_r0030_vds_file, 'r') as vds_file:
    agipd_mask_arr = vds_file['/entry_1/data_1/mask'][95]

agipd_plot_arr = agipd_data_arr * (agipd_mask_arr == 0).astype(float)

agipd_geom.plot_data_fast(agipd_plot_arr, vmin=0, vmax=3000)

Sometimes a mask from the offline calibration is not enough and an additional, external, mask may be required.

## Lanthanum hexaboride powder scattering

Before proceeding further with the analysis of data from the lysozyme crystals scattering let's take a look at the data from scattering X-rays on lanthanum hexaboride (LaB6) powder:

![LaB6](img/LaB6.png)

These data have been measured with a bit different detector - LPD1M:

In [ ]:
lpd_geom_file = "../geom/lpd1m_p700000_r0299_v01.geom"
lpd_geom = LPD_1MGeometry.from_crystfel_geom(lpd_geom_file)
lpd_geom.inspect()

This detector require some additional mask for bad detector pixels which we will load from an external file:

In [ ]:
lpd_ext_mask_file = "../mask/mask_p700000_r0299_total_v2.h5"
with h5py.File(lpd_ext_mask_file, 'r') as h5m_in:
    lpd_ext_mask = h5m_in['/entry_1/data_1/mask'][:]

lpd_geom.plot_data_fast(lpd_ext_mask.astype(float))

To smooth random detector background we will take an average of 20 data frames:

In [ ]:
p700000_r0299_vds_file = "../data/p700000_r0299_proc.cxi"
with h5py.File(p700000_r0299_vds_file, 'r') as vds_file:
    lpd_data_arr = vds_file['/entry_1/data_1/data'][:20]
    lpd_mask_arr = (vds_file['/entry_1/data_1/mask'][:20] == 0).astype(float)

lpd_data_mask_arr = lpd_data_arr * lpd_mask_arr
lpd_data_mean = np.mean(lpd_data_mask_arr, axis=0) * (lpd_ext_mask == 0).astype(float)

lpd_geom.plot_data_fast(lpd_data_mean, vmin=0, vmax=4000)

From the image above one can see that one of the detector pannels, on the bottom right, has been switched off during the experiment.

Our data, as expected, appear as a set of co-centric rings with the centre at our X-ray beam. We are going to transform it to the polar coordinates with a help of `pyFAI` library:

In [ ]:
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator

As a reminder, [polar coordinate system](https://en.wikipedia.org/wiki/Polar_coordinate_system) is a two-dimensional coordinate system in which each point on a plane is determined by a distance from a reference point and an angle from a reference direction:

![polar_to_cartesian.png](img/polar_to_cartesian.png)

In our case a reference point is detector centre which corresponds to the X-ray beam position.

In [ ]:
lpd_frame_data, lpd_frame_centre = lpd_geom.position_modules(lpd_data_mean)

integrator = AzimuthalIntegrator(
    dist=lpd_geom.metadata['crystfel']['clen'],
    pixel1=lpd_geom.pixel_size,
    pixel2=lpd_geom.pixel_size,
    poni1=lpd_frame_centre[0] * lpd_geom.pixel_size,
    poni2=lpd_frame_centre[1] * lpd_geom.pixel_size
)

radius = ((lpd_frame_data.shape[0]/2)**2 + (lpd_frame_data.shape[1]/2)**2)**(1/2)
azimuth_bins = radius * (lpd_frame_data.shape[0]/lpd_frame_data.shape[1])

lpd_integrate2d_result = integrator.integrate2d(
    lpd_frame_data,
    int(radius),
    int(azimuth_bins),
    unit='r_m',
    dummy=np.nan,
    method='cython'
)

lpd_integrate_2d = lpd_integrate2d_result.intensity
lpd_integrate_r = lpd_integrate2d_result.radial
lpd_integrate_phi = lpd_integrate2d_result.azimuthal

To visualize our transformed data we will use a function:

In [ ]:
def draw_polar_array(img_arr, invert_x: bool=True):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(img_arr)
    ax.set_xlabel('r, pixels')
    ax.set_ylabel('theta, pixels')
    if invert_x:
        ax.invert_xaxis()
    ax.invert_yaxis()

In [ ]:
draw_polar_array(np.clip(lpd_integrate_2d, 0, 4000), invert_x=False)

If detector centre have been estimated properly and detector pannels are perfectly aligned our scattering data in polar coordinates should appear as a set of straight vertical lines.

Let's take an average over the $\theta$ axis:

In [ ]:
lpd_integrate_1d_r_mean = np.nanmean(lpd_integrate_2d, axis=0)

In [ ]:
int1d_x = lpd_integrate_r
int1d = lpd_integrate_1d_r_mean

plt.figure(figsize=(18, 4))
plt.plot(int1d_x, int1d)
plt.xlabel('r, m', fontsize=18)
plt.ylabel('intensity', fontsize=18)
plt.yscale('log')

From the structure of our powder sample we know that diffraction rings should be at the resolution values of $4.16Å$, $2.94Å$, $2.40Å$, $2.08Å$, $1.86Å$, ...

In [ ]:
lab6_rings_A = [4.157, 2.939, 2.400, 2.078, 1.859]

To transform these resolution values $d$ to values in meters $r$ we have to know the exact sample-to-detector distance $L$ and X-rays energy $E$ and use next equations:

![rings_r](img/rings_r.png)

![wave_length](img/wave_length.png)

With this we can write a function:

In [ ]:
def get_det_r(clen, d, wave_E):
    """
    clen - in m
    d - in A
    wave_E - in eV
    
    r - in m
    """
    hc = 1.23984198
    wave_len_A = (hc / wave_E) * 10000

    l_div_d = wave_len_A / d
    res = clen * np.sqrt((1 / (1 - l_div_d**2/2)**2) - 1)
    return res

Knowing from the experiment that the sample-to-detector distance was $39.5cm$ and X-rays energy $9425eV$ we can check how does our distribution allign to the prediction:

In [ ]:
clen = 0.395
wave_E = 9425
lab6_rings_r = [get_det_r(clen, d, wave_E) for d in lab6_rings_A]

plt.figure(figsize=(18, 4))
plt.plot(int1d_x, int1d)
for lab6_r in lab6_rings_r:
    plt.axvline(lab6_r, color='r', linestyle=':', alpha=0.8)
plt.xlabel('r, m', fontsize=18)
plt.ylabel('intensity', fontsize=18)
plt.yscale('log')

## Back to the serial crystallography - using CrystFEL suite

[CrystFEL](https://www.desy.de/~twhite/crystfel/) is a suite of programs for processing Serial Femtosecond Crystallography diffraction data. It comprises programs for indexing and integrating diffraction patterns, scaling and merging intensities, calculating figures of merit, and many more.

To start CrystFEL as a GUI one should simply run in the terminal within VISA image:

```shell
crystfel
```

- load the data from `~/hercules_SFX_2023/data/p700000_r0030_proc.cxi`
- select geometry from `~/hercules_SFX_2023/geom/agipd_p700000_r0030_v01.geom`
- in menu `Tools -> Jump to frame` choose frame number `//95`
- scroll and vary the scale on the right fo better data visibility
- in `Peak detection` tab:
  - select 'peakfinder8'
  - change minimum number of pixels to 1
- in `Index this frame` tab:
  - select cell file from `~/hercules_SFX_2023/cell/hewl.cell`
  - chose only `XGANDALF` as indexing method
  - deselect all additional options

This should provide some guidance on how to start using the GUI - feel free to modify any parameters or select a different frame. A list of interesting frames can be found at:

`hercules_SFX_2023/xwiz/t_01/indexed_p700000_r0030_n100.lst`

![visa_crystfel_gui](img/visa_crystfel_gui.png)

## EXtra-xwiz

CrystFEL also provide a command line interface `indexamajig` which allows to run the analysis from the terminal. To make use of this interface more convenient, reproducible and automatic we provide a data analysis pipeline `EXtra-xwiz`:

![xwiz_schema](img/xwiz_schema.png)

EXtra-xwiz allows to run analysis by setting peak finding, indexing and reflections scaling and merging parameters in a single configuration file. It handles distribution of computations over nodes on the SLURM cluster for faster computations and allows to handle automatically some of the processing steps specific for EuXFEL, such as generating virtual dataset files or splitting data frames into saparate datasets in the experiments with sample illumination by a pump laser.

Let's take a look on the example of xwiz configuration file, for this run in the terminal:

```shell
cd ~/hercules_SFX_2023/xwiz/t_01
vim xwiz_conf.toml
```

(to exit `vim` simply press `Esc` and type `:q`)

In [ ]:
xwiz_conf_file_t01 = "../xwiz/t_01/xwiz_conf.toml"
with open(xwiz_conf_file_t01, 'r') as fin:
    for line in fin:
        print(line, end='')

Configuration file is split into sections:
- `[data]` contains information related to the input detector data, such as proposal and run number. It also has options for selecting a subset of frames, in this case we are going to use a list of preselected interesting frames `frames_list_file = "indexed_p700000_r0030_n100.lst"` - indexing rate in our test example should be ~100%, while with processing all data in this run it is in the order of 10%.
- `[crystfel]` allows to specify different versions of CrystFEL suite. Out visa image contain only the newest to date version `0.10.2` which we specify as `0.10.2_visa`.
- `[geom]` - path to the same AGIPD geometry file we have used before.
- `[slurm]` - configuration of SLURM cluster computations distribution. We don't have access to the cluster and therefore will use `partition = 'local'`.
- `[proc_coarse]` - parameters for peak search and indexing diffraction patterns. It is important to set a single thread computations with `n_cores = 1` otherwise processes in the visa image will get stuck. On Maxwell cluster we usually set `n_cores = -1`, which means using all available threads (usually 70) on each SLURM node.
- `[unit_cell]` - path to the file with unit cell parameters.
- `[merging]` - scaling and marging parameters for CrystFEL's `partialator` tool.

To start processing with the xwiz pipeline simply execute in the same folder:

```shell
xwiz-workflow -a -d
```

After the processing it will generate a `p700000_r0030.stream` with CrystFEL output, a `partialator` folder with `.hkl` files and figures-of-merit tables, and a `p700000_r0030.summary` file with some summary of the processing results, for example:

```shell
vim ~/hercules_SFX_2023/xwiz/t_03/p700000_r0030.summary
```

In [ ]:
xwiz_summary_file_t03 = "../xwiz/t_03/p700000_r0030.summary"
with open(xwiz_summary_file_t03, 'r') as fin:
    summary_lines = fin.readlines()
    print("...")
    for line in summary_lines[49:]:
        print(line, end='')

Of course in our example figures of merit will be much worse since we have processed only 100 crystals. To process more crystalls with a bit better time performance we can first write interesting frames into a new HD5 file with the script prepared in `~/hercules_SFX_2023/xwiz/t_02/`:

```shell
cd ~/hercules_SFX_2023/xwiz/t_02
./prep_h5data.py
```

This should generate a `indexed_p700000_r0030_vds.h5` file. In the `xwiz_conf.toml` you can notice a difference in the `[data]` section:

In [ ]:
xwiz_conf_file_t02 = "../xwiz/t_02/xwiz_conf.toml"
with open(xwiz_conf_file_t02, 'r') as fin:
    conf_lines_t02 = fin.readlines()
    for line in conf_lines_t02[:5]:
        print(line, end='')
    print("...")

There `vds_names = ["indexed_p700000_r0030_vds.h5"]` specifies our generated HD5 file, and `frames_range = {end = -1}` allows to select a subset of frames from it, with `end = -1` meaning to take all available frames from the file.

CrystFEL output stream contains a lot of useful information and can be used, for example, to plot the distribution of cell parameters. For this run in the terminal within the visa image:

```shell
cell_explorer ~/hercules_SFX_2023/xwiz/t_01/p700000_r0030.stream
```

(or change to the stream in `t_02` or `t_03` example folder, although `t_03` may take long time to read all data)
Distributions for `t_01` and `t_02` will be pretty low in statistics, but for `t_03` they should look like:

![cell_expl_xwiz_t03](img/cell_expl_xwiz_t03.png)

To fit the six unit cell parameters:
 - Adjust the scale for each parameter with the mouse scrolling while the pointer is over the distribution figure
 - Drag the mouse with shift left-button to mark an interval around the peak, for all 6 parameters
 - Select `Tools -> Fit cell` from the menu

We can also use the CrystFEL stream to compute a sum of distributions of the peaks intensity (or the number of peaks per detector pixel, as depicted bellow). Hot to perform the computation is covered in the extra material at `~hercules_SFX_2023/jup/extra/hercules_SFX_sum_peaks.ipynb`, here will just load the result:

In [ ]:
agipd_sum_peaks_file = "extra/sum_peaks_p700000_r0030.h5"
with h5py.File(agipd_sum_peaks_file, 'r') as h5m_in:
    agipd_intensity_sum = h5m_in['/sum_peaks/intensity_sum'][:]
    agipd_peaks_num = h5m_in['/sum_peaks/peaks_num'][:]

In [ ]:
agipd_geom.plot_data_fast(agipd_peaks_num, vmin=0, vmax=70)

As an exercise - try to transform this image into the polar coordinates :)

Structure factor intensities in the format of an `.hkl` file are stored by the pipeline in the `partialator` folder. Further processing to obtain electron density map and reconstruct the protein are outside of the scope of this tutorial, but [here](https://www.rcsb.org/3d-view/6ABN?preset=electronDensityMaps) you can see some 3D view of the Lysozyme, as on the beginning of the notebook.

![lysozyme_3D](img/lysozyme_3D.png)